This is an example of a simple CNN developed, trained and utilized

AI was used to help generate the codebase

Note: Make sure that the tensorflow package is installed in your device.

In [10]:
# Lib imports
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
import numpy as np

In [11]:
# DATASET DIRECTORY CONFIGURATION
# Download and unzip the dataset from Kaggle, set the directory paths accordingly.
train_dir = "D:/CODE/25-26/CS 3B/CHRISTIAN_PAULO_PILLADO/Exercise_6/train"  # e.g. './muffin-vs-chihuahua/train'
test_dir = "D:/CODE/25-26/CS 3B/CHRISTIAN_PAULO_PILLADO/Exercise_6/test"    # e.g. './muffin-vs-chihuahua/test'

In [12]:
# IMAGE PARAMETERS
# Used to resize the input images, also will determine the input size of your input layer.
IMG_SIZE = (128, 128)
BATCH_SIZE = 32

In [13]:
# DATA PREPROCESSING & AUGMENTATION
# Optional but recommended for image processing tasks, especially with limited data.
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    validation_split=0.2
)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='training'
)
val_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='validation'
)
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    shuffle=False
)

Found 3788 images belonging to 2 classes.
Found 945 images belonging to 2 classes.
Found 1184 images belonging to 2 classes.


In [14]:
# SIMPLE CNN MODEL ARCHITECTURE

# Some modifications are applied
initial_learning_rate = 0.001
# We are combining ExponentialDecay with Adam optimizer for better learning rate management
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate,
    decay_steps=10000,
    decay_rate=0.9,
    staircase=True
)

# Create the optimizer with the learning rate schedule
optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)

# Applied dropout layers to reduce overfitting
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3)),
    layers.MaxPooling2D(2, 2),
    # layers.Dropout(0.25),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D(2, 2),
    # layers.Dropout(0.25),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D(2, 2),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    # layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')
])

In [15]:
# Configure the model optimizers, loss function, and metrics
# model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy']) # old
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

In [16]:
# TRAINING THE CNN
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=val_generator
)

Epoch 1/10
 38/119 ━━━━━━━━━━━━━━━━━━━━ 1:10 865ms/step - accuracy: 0.5825 - loss: 0.6870

KeyboardInterrupt: 

In [ ]:
# EVALUATE THE MODEL
test_loss, test_acc = model.evaluate(test_generator)
print(f"Test Accuracy: {test_acc}")

37/37 ━━━━━━━━━━━━━━━━━━━━ 13s 358ms/step - accuracy: 0.9096 - loss: 0.2137
Test Accuracy: 0.9096283912658691


In [ ]:
# SAVE THE MODEL
model.save('muffin_vs_chihuahua_cnn.h5')

In [ ]:
# SIMPLE INFERENCE SCRIPT
from tensorflow.keras.preprocessing import image

def predict_image(img_path, model_path='muffin_vs_chihuahua_cnn.h5'):
    model = tf.keras.models.load_model(model_path)
    img = image.load_img(img_path, target_size=IMG_SIZE)
    img_array = image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    pred = model.predict(img_array)[0,0]
    label = "Chihuahua" if pred >= 0.5 else "Muffin"
    print(f"Prediction: {label} (confidence: {pred:.2f})")


In [ ]:
# Example usage:
predict_image("D:/CODE/25-26/CS 3B/CHRISTIAN_PAULO_PILLADO/Exercise_6/predict/img1.jpg")
predict_image("D:/CODE/25-26/CS 3B/CHRISTIAN_PAULO_PILLADO/Exercise_6/predict/img2.jpg")
predict_image("D:/CODE/25-26/CS 3B/CHRISTIAN_PAULO_PILLADO/Exercise_6/predict/img3.jpg")
predict_image("D:/CODE/25-26/CS 3B/CHRISTIAN_PAULO_PILLADO/Exercise_6/predict/img4.jpg")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 141ms/step


Prediction: Muffin (confidence: 0.03)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 145ms/step


Prediction: Chihuahua (confidence: 0.64)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 156ms/step


Prediction: Chihuahua (confidence: 0.99)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 155ms/step
Prediction: Chihuahua (confidence: 1.00)


Improvement of CNN Architecture

Improved Model Architecture with Regularization and Dropout

In [ ]:
# IMPROVED CNN MODEL ARCHITECTURE with Regularization and Dropout
from tensorflow.keras import regularizers

# Lower learning rate for more stable training
initial_learning_rate = 0.0001
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate,
    decay_steps=1000,
    decay_rate=0.96,
    staircase=True
)

optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)

# Model with L2 regularization and dropout
model_improved = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', 
                  kernel_regularizer=regularizers.l2(0.001),
                  input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3)),
    layers.MaxPooling2D(2, 2),
    layers.Dropout(0.25),
    
    layers.Conv2D(64, (3, 3), activation='relu',
                  kernel_regularizer=regularizers.l2(0.001)),
    layers.MaxPooling2D(2, 2),
    layers.Dropout(0.25),
    
    layers.Conv2D(128, (3, 3), activation='relu',
                  kernel_regularizer=regularizers.l2(0.001)),
    layers.MaxPooling2D(2, 2),
    layers.Dropout(0.25),
    
    layers.Flatten(),
    layers.Dense(128, activation='relu',
                 kernel_regularizer=regularizers.l2(0.001)),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')
])

model_improved.compile(optimizer=optimizer, 
                      loss='binary_crossentropy', 
                      metrics=['accuracy'])
model_improved.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_3 (Conv2D)               │ (None, 126, 126, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 63, 63, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 63, 63, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 61, 61, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 30, 30, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 30, 30, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 28, 28, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 14, 14, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ (None, 14, 14, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 128)            │     3,211,392 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_7 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,304,769 (12.61 MB)

 Trainable params: 3,304,769 (12.61 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# TRAINING THE IMPROVED CNN
history_improved = model_improved.fit(
    train_generator,
    epochs=10,  # more epochs with regularization
    validation_data=val_generator
)

Epoch 1/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 365s 3s/step - accuracy: 0.5488 - loss: 1.0367 - val_accuracy: 0.7111 - val_loss: 0.9668
Epoch 2/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 126s 1s/step - accuracy: 0.6679 - loss: 0.8853 - val_accuracy: 0.7725 - val_loss: 0.8065
Epoch 3/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 131s 1s/step - accuracy: 0.7402 - loss: 0.7816 - val_accuracy: 0.8021 - val_loss: 0.7439
Epoch 4/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 127s 1s/step - accuracy: 0.7584 - loss: 0.7288 - val_accuracy: 0.7958 - val_loss: 0.6990
Epoch 5/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 130s 1s/step - accuracy: 0.7732 - loss: 0.6869 - val_accuracy: 0.8106 - val_loss: 0.6824
Epoch 6/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 129s 1s/step - accuracy: 0.7941 - loss: 0.6437 - val_accuracy: 0.8159 - val_loss: 0.6352
Epoch 7/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 123s 1s/step - accuracy: 0.7999 - loss: 0.6178 - val_accuracy: 0.7820 - val_loss: 0.6627
Epoch 8/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 122s 1s/step - accuracy: 0.8054 - loss: 0.6022 - val_accu

In [ ]:
# EVALUATE THE IMPROVED MODEL
test_loss_improved, test_acc_improved = model_improved.evaluate(test_generator)
print(f"\n=== IMPROVED MODEL RESULTS ===")
print(f"Test Accuracy: {test_acc_improved:.4f}")
print(f"Test Loss: {test_loss_improved:.4f}")

# Compare with original if you have it
print(f"\n=== COMPARISON ===")
print(f"Original Test Accuracy: {test_acc:.4f}" if 'test_acc' in locals() else "Original not trained")
print(f"Improved Test Accuracy: {test_acc_improved:.4f}")

37/37 ━━━━━━━━━━━━━━━━━━━━ 46s 1s/step - accuracy: 0.7677 - loss: 0.6424

=== IMPROVED MODEL RESULTS ===
Test Accuracy: 0.7677
Test Loss: 0.6424

=== COMPARISON ===
Original not trained
Improved Test Accuracy: 0.7677


In [17]:
# SAVE THE IMPROVED MODEL
model_improved.save('exercise_6_trained_model_improved.h5')
print("Improved model saved as 'exercise_6_trained_model_improved.h5'")

Improved model saved as 'exercise_6_trained_model_improved.h5'


In [18]:
# IMPROVED INFERENCE SCRIPT
def predict_image_improved(img_path, model):
    img = image.load_img(img_path, target_size=IMG_SIZE)
    img_array = image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    pred = model.predict(img_array, verbose=0)[0,0]
    label = "Muffin" if pred >= 0.5 else "Chihuahua"
    confidence = pred if pred >= 0.5 else (1 - pred)
    return label, confidence, pred

# RUN 1: Predictions with improved model
print("=== RUN 1: IMPROVED MODEL PREDICTIONS ===")
test_images = [
    "D:/CODE/25-26/CS 3B/CHRISTIAN_PAULO_PILLADO/Exercise_6/predict/img1.jpg",
    "D:/CODE/25-26/CS 3B/CHRISTIAN_PAULO_PILLADO/Exercise_6/predict/img2.jpg",
    "D:/CODE/25-26/CS 3B/CHRISTIAN_PAULO_PILLADO/Exercise_6/predict/img3.jpg",
    "D:/CODE/25-26/CS 3B/CHRISTIAN_PAULO_PILLADO/Exercise_6/predict/img4.jpg"
]

for i, img_path in enumerate(test_images, 1):
    label, confidence, raw_pred = predict_image_improved(img_path, model_improved)
    print(f"Image {i}: {label} (Confidence: {confidence:.2%}, Raw: {raw_pred:.4f})")

# RUN 2: Second run with same model
print("\n=== RUN 2: IMPROVED MODEL PREDICTIONS (Second Run) ===")
for i, img_path in enumerate(test_images, 1):
    label, confidence, raw_pred = predict_image_improved(img_path, model_improved)
    print(f"Image {i}: {label} (Confidence: {confidence:.2%}, Raw: {raw_pred:.4f})")

=== RUN 1: IMPROVED MODEL PREDICTIONS ===


NameError: name 'image' is not defined